In [1]:
#Eager Mode
import tensorflow as tf

tf.enable_eager_execution()    
import tensorflow.contrib.eager as tfe
print("Eager execution: {}".format(tf.executing_eagerly())) 

Eager execution: True


In [2]:
#import tensorflow as tf
#tf.enable_eager_execution()

from os import listdir
import os
import matplotlib.pyplot as plt
import numpy as np
import time
from PIL import Image
from tensorflow.python.keras import models
from scipy.optimize import fmin_l_bfgs_b
import urllib
from tensorflow.python.keras.preprocessing import image as kp_image

from IPython import display
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, BatchNormalization, Dropout,Add

In [3]:
class TransformNet(tf.keras.Model):
  def __init__(self):
    super(TransformNet, self).__init__()
    # encode block
    self.block1_conv1 = Conv2D(filters=32, kernel_size=(9,9), strides=1, padding="same")
    self.block1_batchnorm1 = BatchNormalization(axis=3)
    self.block1_conv2 = Conv2D(filters=64, kernel_size=(3,3), strides=2, padding="same")
    self.block1_batchnorm2 = BatchNormalization(axis=3)
    self.block1_conv3 = Conv2D(filters=128, kernel_size=(3,3), strides=2, padding="same")
    self.block1_batchnorm3 = BatchNormalization(axis=3)
    # residual net block
    self.block2_conv1 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm1 = BatchNormalization(axis=3)
    self.block2_conv2 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm2 = BatchNormalization(axis=3)
    self.block2_add1 = Add()
    self.block2_conv3 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm3 = BatchNormalization(axis=3)
    self.block2_conv4 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm4 = BatchNormalization(axis=3)
    self.block2_add2 = Add()
    self.block2_conv5 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm5 = BatchNormalization(axis=3)
    self.block2_conv6 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm6 = BatchNormalization(axis=3)
    self.block2_add3 = Add()
    self.block2_conv7 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm7 = BatchNormalization(axis=3)
    self.block2_conv8 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm8 = BatchNormalization(axis=3)
    self.block2_add4 = Add()
    self.block2_conv9 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm9 = BatchNormalization(axis=3)
    self.block2_conv10 = Conv2D(filters=128, kernel_size=(3,3), padding="same")
    self.block2_batchnorm10 = BatchNormalization(axis=3)
    self.block2_add5 = Add()
    # decode block
    self.block3_conv1transpose = Conv2DTranspose(filters=64, kernel_size=(3,3), strides=2, padding="same")
    self.block3_batchnorm1 = BatchNormalization(axis=3)
    self.block3_conv2transpose = Conv2DTranspose(filters=32, kernel_size=(3,3), strides=2, padding="same")
    self.block3_batchnorm2 = BatchNormalization(axis=3)
    self.block3_conv3transpose = Conv2D(filters=3, kernel_size=(9,9), strides=1, padding="same")
    self.block3_batchnorm3 = BatchNormalization(axis=3)
    
  def call(self, x, training=True):
    # encode block
    x = tf.reshape(x,(-1,512,512,3))
    x = self.block1_conv1(x)
    x = tf.nn.relu(x)
    x = self.block1_batchnorm1(x, training=training)
    x = self.block1_conv2(x)
    x = tf.nn.relu(x)
    x = self.block1_batchnorm2(x, training=training)
    x = self.block1_conv3(x)
    x = tf.nn.relu(x)
    x = self.block1_batchnorm3(x, training=training)
    
    # residual block
    x1 = x
    x = self.block2_conv1(x)
    x = self.block2_batchnorm1(x,training=training)
    x = tf.nn.relu(x)
    x = self.block2_conv2(x)
    x = self.block2_batchnorm2(x,training=training)
    x = self.block2_add1([x, x1])
    x1 = x
    x = self.block2_conv3(x)
    x = self.block2_batchnorm3(x,training=training)
    x = tf.nn.relu(x)
    x = self.block2_conv4(x)
    x = self.block2_batchnorm4(x,training=training)
    x = self.block2_add2([x, x1])
    x1 = x
    x = self.block2_conv5(x)
    x = self.block2_batchnorm5(x,training=training)
    x = tf.nn.relu(x)
    x = self.block2_conv6(x)
    x = self.block2_batchnorm6(x,training=training)
    x = self.block2_add3([x, x1])
    x1 = x
    x = self.block2_conv7(x)
    x = self.block2_batchnorm7(x,training=training)
    x = tf.nn.relu(x)
    x = self.block2_conv8(x)
    x = self.block2_batchnorm8(x,training=training)
    x = self.block2_add4([x, x1])
    x1 = x
    x = self.block2_conv9(x)
    x = self.block2_batchnorm9(x,training=training)
    x = tf.nn.relu(x)
    x = self.block2_conv10(x)
    x = self.block2_batchnorm10(x,training=training)
    x = self.block2_add5([x, x1])
    
    # decode block
    x = self.block3_conv1transpose(x)
    x = tf.nn.relu(x)
    x = self.block3_batchnorm1(x,training=training)
    x = self.block3_conv2transpose(x)
    x = tf.nn.relu(x)
    x = self.block3_batchnorm2(x,training=training)
    x = self.block3_conv3transpose(x)
    x = (tf.nn.tanh(x)+1)*127.5  
    #so as to input vgg16, this idea comes from 
    # https://github.com/malhagi/tensorflow-fast-neuralstyle
    return x

In [4]:
def load_model():
    transform_net=TransformNet()
    # Initialize somehow!
    _=transform_net(tfe.Variable(np.zeros((1,512,512,3),dtype=np.float32)), training=True)
    web='http://www.columbia.edu/~hl3099/aml/beta_model.h5' 
    urllib.request.urlretrieve(web,'beta_model.h5')
    transform_net.load_weights('beta_model.h5',by_name=False)
    return transform_net

In [5]:
transform_net=load_model()

In [6]:
def style_transfer_test_data(img,width,height):
  
    # read data
    #test_image = Image.open(i)
    img=cv2.resize(img, (width, height)) 
    test_image = np.array(img,dtype='float32')
    test_image = kp_image.img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    test_image /= 255
    #test_images[0,:,:,:]=np.asarray(test_image, dtype='float32')
  
    # predictions
    test_img=transform_net(test_image,training=False)
    test_img=test_img.numpy()
    test_img=test_img[0,:,:,:]
    return np.clip(test_img,0,255).astype('uint8')

In [21]:
import cv2

video_capture = cv2.VideoCapture(0)
count = 0
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Draw a rectangle around the faces
    #for (x, y, w, h) in faces:
    #    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    with tf.device('/gpu:0'):
        frame=style_transfer_test_data(frame,512,512)
    
    # Display the resulting frame
    cv2.imshow('Video', frame[:,:,::-1])
    #cv2.imwrite("frame%d.jpg" % count, frame)
    
    #content_image_path = "frame%d.jpg" % count
    #create white noise image and load target images
    #content_image,_= load_and_preprocess_img(content_image_path,height,width,means)
    #style_image,_= load_and_preprocess_img(style_image_path,height,width,means)
    #best_loss,best_img=transfer_style(height,width,init_way,means,layers_content,layers_style,
    #                                  content_weight,style_weight,variation_weight,beta,iterations)    
    count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()